In [ ]:
import csv
from datetime import datetime

INVENTARIO_FILE = "inventario_componentes.csv"

# Inicializa el archivo si no existe
def inicializar_archivo():
    try:
        with open(INVENTARIO_FILE, mode='x', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([
                "ID", "Nombre", "Fabricante", "Categoria", "Cantidad", "Fecha_Ultima_Entrada"
            ])
    except FileExistsError:
        pass

# Cargar datos de ejemplo para pruebas
def cargar_datos_ejemplo():
    datos = [
        ["C001", "Arduino UNO", "Arduino", "Microcontrolador", 25, "2025-10-01"],
        ["C002", "Sensor de temperatura LM35", "Texas Instruments", "Sensor", 8, "2025-09-15"],
        ["C003", "ESP32 DevKit", "Espressif", "Microcontrolador", 5, "2025-08-20"],
        ["C004", "Resistor 220 Ohm", "Vishay", "Pasivo", 100, "2025-10-10"],
        ["C005", "Capacitor 10uF", "Murata", "Pasivo", 12, "2025-10-05"],
        ["C006", "Sensor de humedad DHT11", "Aosong", "Sensor", 3, "2025-07-30"],
        ["C007", "Módulo Bluetooth HC-05", "Waveshare", "Comunicación", 9, "2025-09-01"],
        ["C008", "OLED 0.96 pulgadas", "Adafruit", "Pantalla", 15, "2025-10-12"],
        ["C009", "L293D Driver de motor", "STMicroelectronics", "Controlador", 6, "2025-08-10"],
        ["C010", "Batería Li-Ion 18650", "Samsung", "Energía", 20, "2025-10-18"]
    ]

    with open(INVENTARIO_FILE, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([
            "ID", "Nombre", "Fabricante", "Categoria", "Cantidad", "Fecha_Ultima_Entrada"
        ])
        writer.writerows(datos)
    print("Base de datos de ejemplo cargada correctamente.")

# Agrega un nuevo componente
def agregar_componente():
    print("\n--- Agregar nuevo componente ---")
    id_parte = input("ID de parte: ")
    nombre = input("Nombre del componente: ")
    fabricante = input("Fabricante/Proveedor: ")
    categoria = input("Categoría: ")
    cantidad = int(input("Cantidad en stock: "))
    fecha_entrada = datetime.today().strftime('%Y-%m-%d')

    with open(INVENTARIO_FILE, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([id_parte, nombre, fabricante, categoria, cantidad, fecha_entrada])
    print("Componente agregado correctamente.")

# Lectura secuencial de todos los registros
def leer_inventario():
    print("\n--- Inventario completo ---")
    try:
        with open(INVENTARIO_FILE, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for i, row in enumerate(reader, start=1):
                print(f"\nComponente #{i}")
                print(f"  ID: {row['ID']}")
                print(f"  Nombre: {row['Nombre']}")
                print(f"  Fabricante: {row['Fabricante']}")
                print(f"  Categoría: {row['Categoria']}")
                print(f"  Cantidad en stock: {row['Cantidad']}")
                print(f"  Fecha última entrada: {row['Fecha_Ultima_Entrada']}")
    except FileNotFoundError:
        print("El archivo de inventario no existe.")

# Listar componentes con stock bajo y sin entrada reciente
def listar_stock_bajo(fecha_inicio, fecha_fin):
    print("\n--- Componentes con stock bajo y sin entrada reciente ---")
    with open(INVENTARIO_FILE, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        encontrados = False
        for row in reader:
            cantidad = int(row["Cantidad"])
            fecha_entrada = datetime.strptime(row["Fecha_Ultima_Entrada"], '%Y-%m-%d')
            if cantidad < 10 and not (fecha_inicio <= fecha_entrada <= fecha_fin):
                print(f"- {row['ID']} | {row['Nombre']} | Stock: {cantidad} | Última entrada: {row['Fecha_Ultima_Entrada']}")
                encontrados = True
        if not encontrados:
            print("No se encontraron componentes que cumplan los criterios.")

# Construir índice por ID para acceso directo
def construir_indice_id():
    indice = {}
    with open(INVENTARIO_FILE, mode='r', encoding='utf-8') as file:
        pos = file.tell()
        encabezado = file.readline()
        while True:
            pos = file.tell()
            linea = file.readline()
            if not linea:
                break
            campos = linea.strip().split(',')
            if campos:
                id_parte = campos[0]
                indice[id_parte] = pos
    return indice

# Buscar componente por ID usando seek()
def buscar_por_id(id_busqueda):
    indice = construir_indice_id()
    if id_busqueda not in indice:
        print(f"No se encontró el componente con ID '{id_busqueda}'.")
        return

    with open(INVENTARIO_FILE, mode='r', encoding='utf-8') as file:
        file.seek(indice[id_busqueda])
        linea = file.readline()
        campos = linea.strip().split(',')
        if len(campos) == 6:
            print("\nComponente encontrado:")
            print(f"  ID: {campos[0]}")
            print(f"  Nombre: {campos[1]}")
            print(f"  Fabricante: {campos[2]}")
            print(f"  Categoría: {campos[3]}")
            print(f"  Cantidad en stock: {campos[4]}")
            print(f"  Fecha última entrada: {campos[5]}")
        else:
            print("Error al leer los datos del componente.")

# Construir índice por categoría
def construir_indice_categoria():
    indice = {}
    with open(INVENTARIO_FILE, mode='r', encoding='utf-8') as file:
        file.readline()
        while True:
            pos = file.tell()
            linea = file.readline()
            if not linea:
                break
            campos = linea.strip().split(',')
            if len(campos) >= 4:
                categoria = campos[3]
                if categoria not in indice:
                    indice[categoria] = []
                indice[categoria].append(pos)
    return indice

# Buscar componentes por categoría usando índice
def buscar_por_categoria(categoria_busqueda, indice_categoria):
    if categoria_busqueda not in indice_categoria:
        print(f"No se encontraron componentes en la categoría '{categoria_busqueda}'.")
        return

    print(f"\nComponentes en categoría '{categoria_busqueda}':")
    with open(INVENTARIO_FILE, mode='r', encoding='utf-8') as file:
        for pos in indice_categoria[categoria_busqueda]:
            file.seek(pos)
            linea = file.readline()
            campos = linea.strip().split(',')
            if len(campos) == 6:
                print(f"- ID: {campos[0]} | Nombre: {campos[1]} | Fabricante: {campos[2]} | Stock: {campos[4]} | Fecha: {campos[5]}")

# Menú principal
if __name__ == "__main__":
    inicializar_archivo()
    while True:
        print("\nOpciones:")
        print("0. Cargar base de datos de ejemplo")
        print("1. Agregar componente")
        print("2. Listar componentes con stock bajo y sin entrada reciente")
        print("3. Leer inventario completo")
        print("4. Buscar componente por ID (acceso directo)")
        print("5. Buscar componentes por categoría (acceso indexado)")
        print("6. Salir")
        opcion = input("Selecciona una opción: ")

        if opcion == "0":
            cargar_datos_ejemplo()
        elif opcion == "1":
            agregar_componente()
        elif opcion == "2":
            fecha_inicio = input("Fecha inicio (YYYY-MM-DD): ")
            fecha_fin = input("Fecha fin (YYYY-MM-DD): ")
            try:
                f_inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')
                f_fin = datetime.strptime(fecha_fin, '%Y-%m-%d')
                listar_stock_bajo(f_inicio, f_fin)
            except ValueError:
                print("Formato de fecha inválido.")
        elif opcion == "3":
            leer_inventario()
        elif opcion == "4":
            id_busqueda = input("Ingrese el ID de parte a buscar: ")
            buscar_por_id(id_busqueda)
        elif opcion == "5":
            categoria = input("Ingrese la categoría a buscar: ")
            indice_categoria = construir_indice_categoria()
            buscar_por_categoria(categoria, indice_categoria)
        elif opcion == "6":
            print("Saliendo del programa.")
            break
        else:
            print("Opción inválida.")
